In [1]:
import numpy as np
import model
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
enc = model.Encoder()
enc.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Dimages (InputLayer)             (None, 32, 32, 1)     0                                            
____________________________________________________________________________________________________
D4xNIN (Conv2D)                  (None, 32, 32, 256)   256         Dimages[0][0]                    
____________________________________________________________________________________________________
D4xNINWS (WScaleLayer)           (None, 32, 32, 256)   1.0         D4xNIN[0][0]                     
____________________________________________________________________________________________________
add_bias_layer_1 (AddBiasLayer)  (None, 32, 32, 256)   256         D4xNINWS[0][0]                   
___________________________________________________________________________________________

In [3]:
G = model.Generator(use_wscale=False)
G.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Glatents (InputLayer)            (None, 4, 4, 256)     0                                            
____________________________________________________________________________________________________
G1a (Conv2D)                     (None, 4, 4, 256)     1048832     Glatents[0][0]                   
____________________________________________________________________________________________________
G1aBN (BatchNormalization)       (None, 4, 4, 256)     1024        G1a[0][0]                        
____________________________________________________________________________________________________
G1aPN (PixelNormLayer)           (None, 4, 4, 256)     0           G1aBN[0][0]                      
__________________

In [5]:
from keras.models import Sequential
Sequential([enc, G])

In [15]:
D = model.Discriminator(resolution=16)

In [16]:
D.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Dimages (InputLayer)             (None, 16, 16, 1)     0                                            
____________________________________________________________________________________________________
D3xNIN (Conv2D)                  (None, 16, 16, 256)   256         Dimages[0][0]                    
____________________________________________________________________________________________________
D3xNINWS (WScaleLayer)           (None, 16, 16, 256)   1.0         D3xNIN[0][0]                     
____________________________________________________________________________________________________
add_bias_layer_73 (AddBiasLayer) (None, 16, 16, 256)   256         D3xNINWS[0][0]                   
___________________________________________________________________________________________

In [3]:
tw = model.replace_batch_norm(G)

In [21]:
enc = model.Encoder()

In [6]:
enc_tw = model.replace_batch_norm(enc)

In [7]:
enc.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Dimages (InputLayer)             (None, 32, 32, 1)     0                                            
____________________________________________________________________________________________________
D4xNIN (Conv2D)                  (None, 32, 32, 256)   256         Dimages[0][0]                    
____________________________________________________________________________________________________
D4xNINWS (WScaleLayer)           (None, 32, 32, 256)   1.0         D4xNIN[0][0]                     
____________________________________________________________________________________________________
add_bias_layer_1 (AddBiasLayer)  (None, 32, 32, 256)   256         D4xNINWS[0][0]                   
___________________________________________________________________________________________